In [225]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [255]:
df = pd.read_csv('fraud_payment_data', sep=',', header=0) 
print(len(df['Sender_Id'].unique()))
print(len(df['Sender_Account'].unique()))
df

59072
75162


,Time_step,Transaction_Id,Sender_Id,Sender_Account,Sender_Country,Sender_Sector,Sender_lob,Bene_Id,Bene_Account,Bene_Country,USD_amount,Label,Transaction_Type
0,2022-03-15 10:24:00,EXCHANGE-10115,JPMC-CLIENT-10098,ACCOUNT-10108,USA,35537.0,CCB,NaN,NaN,NaN,558.43,0,WITHDRAWAL
1,2022-03-15 10:24:00,QUICK-PAYMENT-10116,JPMC-CLIENT-10098,ACCOUNT-10109,USA,15287.0,CCB,CLIENT-10100,ACCOUNT-10106,CANADA,622.78,0,QUICK-PAYMENT
2,2022-03-15 10:24:00,DEPOSIT-CASH-9833,NaN,NaN,NaN,NaN,NaN,JPMC-CLIENT-9812,ACCOUNT-9826,USA,802.54,0,DEPOSIT-CASH
3,2022-03-15 10:24:00,PAY-CHECK-9832,JPMC-CLIENT-9812,ACCOUNT-9825,USA,38145.0,CCB,JPMC-CLIENT-9814,ACCOUNT-9824,USA,989.09,0,PAY-CHECK
4,2022-03-15 10:24:00,DEPOSIT-CHECK-9806,NaN,NaN,NaN,NaN,NaN,JPMC-CLIENT-9789,ACCOUNT-9800,USA,786.78,0,DEPOSIT-CHECK
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498172,2054-09-14 13:17:24,PAY-BILL-2180018,JPMC-CLIENT-2179746,ACCOUNT-2179757,USA,24701.0,CCB,BILL-COMPANY-2179895,ACCOUNT-2179896,USA,159.64,0,MAKE-PAYMENT
1498173,2054-09-15 00:57:24,QUICK-PAYMENT-2181421,JPMC-CLIENT-2181147,ACCOUNT-2181158,USA,4953.0,CCB,CLIENT-2181279,ACCOUNT-2181280,USA,120.07,0,QUICK-PAYMENT
1498174,2054-09-15 01:17:24,PAYMENT-2180021,JPMC-CLIENT-2179746,ACCOUNT-2179757,USA,34784.0,CCB,BILL-COMPANY-2180022,ACCOUNT-2180023,USA,25.32,0,MAKE-PAYMENT
1498175,2054-09-15 12:57:24,QUICK-PAYMENT-2181422,JPMC-CLIENT-2181147,ACCOUNT-2181158,USA,23822.0,CCB,CLIENT-2181279,ACCOUNT-2181280,USA,610.91,0,QUICK-PAYMENT


In [93]:
df['timestamp'] = pd.to_datetime(df['Time_step'])
df=df.drop('Time_step',axis=1)
df=df.drop('Sender_lob',axis=1)
df=df.drop('Sender_Sector',axis=1)
df

,Transaction_Id,Sender_Id,Sender_Account,Sender_Country,Bene_Id,Bene_Account,Bene_Country,USD_amount,Label,Transaction_Type,timestamp
0,EXCHANGE-10115,JPMC-CLIENT-10098,ACCOUNT-10108,USA,NaN,NaN,NaN,558.43,0,WITHDRAWAL,2022-03-15 10:24:00
1,QUICK-PAYMENT-10116,JPMC-CLIENT-10098,ACCOUNT-10109,USA,CLIENT-10100,ACCOUNT-10106,CANADA,622.78,0,QUICK-PAYMENT,2022-03-15 10:24:00
2,DEPOSIT-CASH-9833,NaN,NaN,NaN,JPMC-CLIENT-9812,ACCOUNT-9826,USA,802.54,0,DEPOSIT-CASH,2022-03-15 10:24:00
3,PAY-CHECK-9832,JPMC-CLIENT-9812,ACCOUNT-9825,USA,JPMC-CLIENT-9814,ACCOUNT-9824,USA,989.09,0,PAY-CHECK,2022-03-15 10:24:00
4,DEPOSIT-CHECK-9806,NaN,NaN,NaN,JPMC-CLIENT-9789,ACCOUNT-9800,USA,786.78,0,DEPOSIT-CHECK,2022-03-15 10:24:00
...,...,...,...,...,...,...,...,...,...,...,...
1498172,PAY-BILL-2180018,JPMC-CLIENT-2179746,ACCOUNT-2179757,USA,BILL-COMPANY-2179895,ACCOUNT-2179896,USA,159.64,0,MAKE-PAYMENT,2054-09-14 13:17:24
1498173,QUICK-PAYMENT-2181421,JPMC-CLIENT-2181147,ACCOUNT-2181158,USA,CLIENT-2181279,ACCOUNT-2181280,USA,120.07,0,QUICK-PAYMENT,2054-09-15 00:57:24
1498174,PAYMENT-2180021,JPMC-CLIENT-2179746,ACCOUNT-2179757,USA,BILL-COMPANY-2180022,ACCOUNT-2180023,USA,25.32,0,MAKE-PAYMENT,2054-09-15 01:17:24
1498175,QUICK-PAYMENT-2181422,JPMC-CLIENT-2181147,ACCOUNT-2181158,USA,CLIENT-2181279,ACCOUNT-2181280,USA,610.91,0,QUICK-PAYMENT,2054-09-15 12:57:24


In [95]:
pd.concat([df.Sender_Account,df.Bene_Account]).value_counts()

ACCOUNT-781705     135
ACCOUNT-1591424    135
ACCOUNT-1768107    133
ACCOUNT-483507     130
ACCOUNT-1189368    126
                  ... 
ACCOUNT-933093       1
ACCOUNT-939535       1
ACCOUNT-934223       1
ACCOUNT-930018       1
ACCOUNT-2180023      1
Name: count, Length: 305429, dtype: int64

In [101]:
df.info()
print('There are ' +str(len(df['Sender_Account'].unique()))+ ' unique sender accounts.')
print()
print('There are ' +str(len(df['Bene_Account'].unique()))+ ' unique beneficiary accounts.')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1498177 entries, 0 to 1498176
Data columns (total 11 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   Transaction_Id    1498177 non-null  object        
 1   Sender_Id         1279291 non-null  object        
 2   Sender_Account    1279291 non-null  object        
 3   Sender_Country    1279291 non-null  object        
 4   Bene_Id           1282284 non-null  object        
 5   Bene_Account      1282284 non-null  object        
 6   Bene_Country      1282284 non-null  object        
 7   USD_amount        1498177 non-null  float64       
 8   Label             1498177 non-null  int64         
 9   Transaction_Type  1498177 non-null  object        
 10  timestamp         1498177 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(8)
memory usage: 125.7+ MB
There are 75162 unique sender accounts.

There are 295032 unique beneficiary

In [105]:
(df.Label==1).sum()
Fraud_Trans=df[df.Label==1]
Fraud_Trans
Fraud_Trans.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30819 entries, 11 to 1498072
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction_Id    30819 non-null  object        
 1   Sender_Id         26698 non-null  object        
 2   Sender_Account    26698 non-null  object        
 3   Sender_Country    26698 non-null  object        
 4   Bene_Id           25683 non-null  object        
 5   Bene_Account      25683 non-null  object        
 6   Bene_Country      25683 non-null  object        
 7   USD_amount        30819 non-null  float64       
 8   Label             30819 non-null  int64         
 9   Transaction_Type  30819 non-null  object        
 10  timestamp         30819 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(8)
memory usage: 2.8+ MB


In [257]:
print('There are ' +str(len(Fraud_Trans['Sender_Account'].unique()))+ ' unique fraudulent sender accounts.')
print('There are ' +str(len(Fraud_Trans['Bene_Account'].unique()))+ ' unique fraudulent beneficiary accounts.')

There are 11897 unique fraudulent sender accounts.
There are 16517 unique fraudulent beneficiary accounts.


In [149]:
# Create a directed graph
G = nx.MultiDiGraph()

# Add nodes (accounts)
# You can add all unique senders and receivers as nodes
dumm1=df[['Sender_Account','Sender_Country']].drop_duplicates()
dumm2=df[['Bene_Account','Bene_Country']].drop_duplicates()

for index, row in dumm1.iterrows():
    if row.Sender_Account!='NaN':
      G.add_node(row['Sender_Account'], country=row['Sender_Country'], Id=df.loc[index]['Sender_Id'])
for index, row in dumm2.iterrows():
    if row.Bene_Account!='NaN':
      G.add_node(row['Bene_Account'], country=row['Bene_Country'],Id=df.loc[index]['Bene_Id'])
# Add edges (transactions) with attributes
for index, row in df.iterrows():
    if row['Sender_Account']=='NaN':
        G.add_edge(row['Bene_Account'], row['Bene_Account'], amount=row['USD_amount'], time=row['timestamp'], Type=row['Transaction_Type'],label=row['Label'], Id=row['Transaction_Id'])
    elif row['Bene_Account']=='NaN':
        G.add_edge(row['Sender_Account'], row['Sender_Account'], amount=row['USD_amount'], time=row['timestamp'], Type=row['Transaction_Type'],label=row['Label'], Id=row['Transaction_Id'])
    else:
        G.add_edge(row['Sender_Account'], row['Bene_Account'], amount=row['USD_amount'], time=row['timestamp'], Type=row['Transaction_Type'],label=row['Label'], Id=row['Transaction_Id'])

In [237]:
## Adding node attributes/features like average incoming/outgoing transaction amount, Fraud label, Fraud precentage.
for node in G.nodes():
    Fraud_len=0
    edges=list(G.in_edges(node,data=True))
    len1=len(edges)
    if len1>0:
       amount=[i[2]['amount'] for i in edges]
       G.nodes[node]["in_avg"]=np.mean(amount)
       Fraud_len+=sum([i[2]["label"] for i in edges])
    else: 
       G.nodes[node]["in_avg"]=0
    edges=list(G.out_edges(node,data=True))
    len2=len(edges)
    if len2>0:
       amount=[i[2]['amount'] for i in edges]
       G.nodes[node]["out_avg"]=np.mean(amount)
       Fraud_len+=sum([i[2]["label"] for i in edges])
    else: 
       G.nodes[node]["out_avg"]=0
    if Fraud_len>0:
        G.nodes[node]["Fraud_Status"]=True
    else:
        G.nodes[node]["Fraud_Status"]=False
    G.nodes[node]["Fraud_percentage"]=Fraud_len/(len1+len2)

In [229]:
from scipy.stats import expon
def amount_prob(mean,amount):
    return 1-expon.cdf(x=amount, scale=mean)

In [239]:
## Function that takes as input a node (sender account), and a graph of transactions, and outputs the smallest number of transactions 
## seperating the sender from an account that had previously been involved in fraud.
def min_path_to_fraud(sender_node,G):
    Fraud_nodes = [node for node, data in G.nodes(data=True) if data.get('Fraud') == True]
    shortest_lengths = []
    for target_node in Fraud_nodes:
        try:
            length = nx.shortest_path_length(G, source=sender_node, target=target_node)
            shortest_lengths.append(length)
        except nx.NetworkXNoPath:
            # Handle cases where no path exists
            pass
    for source_node in Fraud_nodes:
        try:
            length_rev = nx.shortest_path_length(G, source=source_node, target=sender_node)
            shortest_lengths.append(length_rev)
        except: 
            pass
    if shortest_lengths:
        return min(shortest_lengths)
    else:
        return None 